In [118]:
from bs4 import BeautifulSoup

In [119]:
html_doc = open('List_of_postal_codes_of_Canada__M.html',encoding='utf-8',errors='ignore')

In [120]:
soup=BeautifulSoup(html_doc,'html.parser')

In [121]:
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

In [122]:
import pandas as pd

In [123]:
dfs = pd.read_html(str(soup.table))

In [124]:
df=dfs[0]

## The input data consist of three columns: PostalCode, Borough, and Neighborhood as a result of web scraping

In [125]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## In next cell we drop rows where Borough is Not assigned

In [126]:
drop_list=[]

for i in df.index:
    if df.loc[i,'Borough'] == 'Not assigned':
        drop_list.append(i)
        
df.drop(drop_list,axis=0,inplace=True)

In [127]:
df.shape

(210, 3)

## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [128]:
for i in df.index:
    if df.loc[i,'Neighbourhood'] == 'Not assigned':
        print('Found')
        df.loc[i,'Neighbourhood'] = df.loc[i,'Borough']

Found


In [129]:
df['Postcode'].unique()

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A', 'M9A', 'M1B', 'M3B', 'M4B',
       'M5B', 'M6B', 'M9B', 'M1C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C',
       'M1E', 'M4E', 'M5E', 'M6E', 'M1G', 'M4G', 'M5G', 'M6G', 'M1H',
       'M2H', 'M3H', 'M4H', 'M5H', 'M6H', 'M1J', 'M2J', 'M3J', 'M4J',
       'M5J', 'M6J', 'M1K', 'M2K', 'M3K', 'M4K', 'M5K', 'M6K', 'M1L',
       'M2L', 'M3L', 'M4L', 'M5L', 'M6L', 'M9L', 'M1M', 'M2M', 'M3M',
       'M4M', 'M5M', 'M6M', 'M9M', 'M1N', 'M2N', 'M3N', 'M4N', 'M5N',
       'M6N', 'M9N', 'M1P', 'M2P', 'M4P', 'M5P', 'M6P', 'M9P', 'M1R',
       'M2R', 'M4R', 'M5R', 'M6R', 'M7R', 'M9R', 'M1S', 'M4S', 'M5S',
       'M6S', 'M1T', 'M4T', 'M5T', 'M1V', 'M4V', 'M5V', 'M8V', 'M9V',
       'M1W', 'M4W', 'M5W', 'M8W', 'M9W', 'M1X', 'M4X', 'M5X', 'M8X',
       'M4Y', 'M7Y', 'M8Y', 'M8Z'], dtype=object)

In [144]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## More than one neighborhood can exist in one postal code area. In the next cell we club them.

In [153]:
df_new = pd.DataFrame()
df_new['Postcode'] = df['Postcode'].unique()
df_new['Borough'] = pd.np.nan
df_new['Neighbourhood'] = pd.np.nan

for index,val in enumerate(df_new['Postcode']):
    temp_list=[]
    for j in df.index:
        if df.loc[j,'Postcode']==val:
            temp_list.append(df.loc[j,'Neighbourhood'])
            boro_j = j
    df_new.loc[index,'Neighbourhood'] = ', '.join(temp_list)
    df_new.loc[index,'Borough'] = df.loc[boro_j,'Borough']

# First Part of Assignment is completed and output is seen in next cell

In [154]:
df_new.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


### Shape of the dataFrame is shown in next cell

In [155]:
df_new.shape

(103, 3)

## Read Location information from csv file

In [156]:
df2=pd.read_csv('Geospatial_Coordinates.csv')

In [157]:
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [158]:
df2.rename(columns={'Postal Code':'Postcode'},inplace=True)

In [159]:
df2.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [160]:
df3 = pd.merge(df_new,df2,on='Postcode')

## Second Part of Assignment is complete. Result in next cell.

In [166]:
df3.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [162]:
import folium

In [225]:
neighborhood_names = ['Central Toronto','Downtown Toronto','West Toronto','East Toronto']
neighborhoods=pd.DataFrame(columns=df3.columns)
for index,j in enumerate(df3['Borough']):
    if j in neighborhood_names:
        neighborhoods.loc[index] = df3.loc[index]

In [226]:
neighborhoods.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')

In [233]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=12)
map_toronto

In [234]:
# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto